In [2]:
import pandas as pd
import numpy as np
import sklearn
import seaborn
import warnings
warnings.filterwarnings('ignore')
import keras 
import tensorflow as tf


Using TensorFlow backend.


In [3]:
read_data = pd.read_csv('train_data.csv')
read_test = pd.read_csv('test_data.csv')

In [66]:
customer_id = read_test['Customer Id']
read_test = read_test.drop('Customer Id', axis = 1)

In [4]:
read_data.head()

,Customer Id,YearOfObservation,Insured_Period,Residential,Building_Painted,Building_Fenced,Garden,Settlement,Building Dimension,Building_Type,Date_of_Occupancy,NumberOfWindows,Geo_Code,Claim
0,H14663,2013,1.0,0,N,V,V,U,290.0,1,1960.0,.,1053,0
1,H2037,2015,1.0,0,V,N,O,R,490.0,1,1850.0,4,1053,0
2,H3802,2014,1.0,0,N,V,V,U,595.0,1,1960.0,.,1053,0
3,H3834,2013,1.0,0,V,V,V,U,2840.0,1,1960.0,.,1053,0
4,H5053,2014,1.0,0,V,N,O,R,680.0,1,1800.0,3,1053,0


In [5]:
read_data.columns

Index(['Customer Id', 'YearOfObservation', 'Insured_Period', 'Residential',
       'Building_Painted', 'Building_Fenced', 'Garden', 'Settlement',
       'Building Dimension', 'Building_Type', 'Date_of_Occupancy',
       'NumberOfWindows', 'Geo_Code', 'Claim'],
      dtype='object')

In [6]:
read_data.describe()

,YearOfObservation,Insured_Period,Residential,Building Dimension,Building_Type,Date_of_Occupancy,Claim
count,7160.000000,7160.000000,7160.000000,7054.000000,7160.000000,6652.000000,7160.000000
mean,2013.669553,0.909758,0.305447,1883.727530,2.186034,1964.456404,0.228212
std,1.383769,0.239756,0.460629,2278.157745,0.940632,36.002014,0.419709
min,2012.000000,0.000000,0.000000,1.000000,1.000000,1545.000000,0.000000
25%,2012.000000,0.997268,0.000000,528.000000,2.000000,1960.000000,0.000000
50%,2013.000000,1.000000,0.000000,1083.000000,2.000000,1970.000000,0.000000
75%,2015.000000,1.000000,1.000000,2289.750000,3.000000,1980.000000,0.000000
max,2016.000000,1.000000,1.000000,20940.000000,4.000000,2016.000000,1.000000


In [7]:
read_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7160 entries, 0 to 7159
Data columns (total 14 columns):
Customer Id           7160 non-null object
YearOfObservation     7160 non-null int64
Insured_Period        7160 non-null float64
Residential           7160 non-null int64
Building_Painted      7160 non-null object
Building_Fenced       7160 non-null object
Garden                7153 non-null object
Settlement            7160 non-null object
Building Dimension    7054 non-null float64
Building_Type         7160 non-null int64
Date_of_Occupancy     6652 non-null float64
NumberOfWindows       7160 non-null object
Geo_Code              7058 non-null object
Claim                 7160 non-null int64
dtypes: float64(3), int64(4), object(7)
memory usage: 783.2+ KB


In [8]:
def find_null(data):
    col_list,val = [],0
    for k in data.columns:
        val = np.sum(data[k].isnull())
        if val != 0:
            col_list.append(k)
        else:
            pass
    return col_list
null_list = find_null(read_data)
null_list

['Garden', 'Building Dimension', 'Date_of_Occupancy', 'Geo_Code']

In [9]:
read_data['Building Dimension'].mode()[0]

400.0

In [10]:
def fillnan(data, col_list):
    for k in col_list:
        mode_k = data[k].mode()[0]
        data[k].fillna(mode_k, inplace = True)
    return data

data = fillnan(read_data, null_list)

In [67]:
read_test = fillnan(read_test, null_list)

In [11]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7160 entries, 0 to 7159
Data columns (total 14 columns):
Customer Id           7160 non-null object
YearOfObservation     7160 non-null int64
Insured_Period        7160 non-null float64
Residential           7160 non-null int64
Building_Painted      7160 non-null object
Building_Fenced       7160 non-null object
Garden                7160 non-null object
Settlement            7160 non-null object
Building Dimension    7160 non-null float64
Building_Type         7160 non-null int64
Date_of_Occupancy     7160 non-null float64
NumberOfWindows       7160 non-null object
Geo_Code              7160 non-null object
Claim                 7160 non-null int64
dtypes: float64(3), int64(4), object(7)
memory usage: 783.2+ KB


In [12]:
find_null(read_data)

[]

In [13]:
read_data = data

In [14]:
# year of observation
read_data['YearOfObservation'].value_counts()

2012    1858
2013    1811
2014    1373
2015    1075
2016    1043
Name: YearOfObservation, dtype: int64

In [15]:
#building painted
read_data['Building_Painted'].value_counts()

V    5382
N    1778
Name: Building_Painted, dtype: int64

In [16]:
read_data['Building_Fenced'].value_counts()

N    3608
V    3552
Name: Building_Fenced, dtype: int64

In [17]:
read_data['Garden'].value_counts()

O    3609
V    3551
Name: Garden, dtype: int64

In [18]:
read_data['Settlement'].value_counts()

R    3610
U    3550
Name: Settlement, dtype: int64

In [19]:
read_data = pd.get_dummies(read_data, columns =['Settlement', 'Garden','Building_Fenced','Building_Painted'])

In [68]:
read_test = pd.get_dummies(read_test, columns =['Settlement', 'Garden','Building_Fenced','Building_Painted'])

In [20]:
read_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7160 entries, 0 to 7159
Data columns (total 18 columns):
Customer Id           7160 non-null object
YearOfObservation     7160 non-null int64
Insured_Period        7160 non-null float64
Residential           7160 non-null int64
Building Dimension    7160 non-null float64
Building_Type         7160 non-null int64
Date_of_Occupancy     7160 non-null float64
NumberOfWindows       7160 non-null object
Geo_Code              7160 non-null object
Claim                 7160 non-null int64
Settlement_R          7160 non-null uint8
Settlement_U          7160 non-null uint8
Garden_O              7160 non-null uint8
Garden_V              7160 non-null uint8
Building_Fenced_N     7160 non-null uint8
Building_Fenced_V     7160 non-null uint8
Building_Painted_N    7160 non-null uint8
Building_Painted_V    7160 non-null uint8
dtypes: float64(3), int64(4), object(3), uint8(8)
memory usage: 615.4+ KB


In [21]:
read_data['Geo_Code'].value_counts()[:5]

6088     245
33063    137
6083     113
13206     98
31555     87
Name: Geo_Code, dtype: int64

In [22]:
# variable description
description = pd.read_csv('VariableDescription.csv')
description

,Variable,Description
0,Customer Id,Identification number for the Policy holder
1,YearOfObservation,year of observation for the insured policy
2,Insured_Period,duration of insurance policy in Olusola Insura...
3,Residential,is the building a residential building or not
4,Building_Painted,"is the building painted or not (N-Painted, V-N..."
5,Building_Fenced,"is the building fence or not (N-Fenced, V-Not ..."
6,Garden,building has garden or not (V-has garden; O-no...
7,Settlement,Area where the building is located. (R- rural ...
8,Building Dimension,Size of the insured building in m2
9,Building_Type,"The type of building (Type 1, 2, 3, 4)"


In [23]:
read_data['Claim'].value_counts()

0    5526
1    1634
Name: Claim, dtype: int64

In [24]:
read_data = read_data.drop(['Geo_Code'], axis = 1)

In [69]:
read_test = read_test.drop(['Geo_Code'], axis = 1)

In [25]:
read_data.head()

,Customer Id,YearOfObservation,Insured_Period,Residential,Building Dimension,Building_Type,Date_of_Occupancy,NumberOfWindows,Claim,Settlement_R,Settlement_U,Garden_O,Garden_V,Building_Fenced_N,Building_Fenced_V,Building_Painted_N,Building_Painted_V
0,H14663,2013,1.0,0,290.0,1,1960.0,.,0,0,1,0,1,0,1,1,0
1,H2037,2015,1.0,0,490.0,1,1850.0,4,0,1,0,1,0,1,0,0,1
2,H3802,2014,1.0,0,595.0,1,1960.0,.,0,0,1,0,1,0,1,1,0
3,H3834,2013,1.0,0,2840.0,1,1960.0,.,0,0,1,0,1,0,1,0,1
4,H5053,2014,1.0,0,680.0,1,1800.0,3,0,1,0,1,0,1,0,0,1


In [26]:
read_data = read_data.drop('Customer Id', axis = 1)

In [27]:
read_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7160 entries, 0 to 7159
Data columns (total 16 columns):
YearOfObservation     7160 non-null int64
Insured_Period        7160 non-null float64
Residential           7160 non-null int64
Building Dimension    7160 non-null float64
Building_Type         7160 non-null int64
Date_of_Occupancy     7160 non-null float64
NumberOfWindows       7160 non-null object
Claim                 7160 non-null int64
Settlement_R          7160 non-null uint8
Settlement_U          7160 non-null uint8
Garden_O              7160 non-null uint8
Garden_V              7160 non-null uint8
Building_Fenced_N     7160 non-null uint8
Building_Fenced_V     7160 non-null uint8
Building_Painted_N    7160 non-null uint8
Building_Painted_V    7160 non-null uint8
dtypes: float64(3), int64(4), object(1), uint8(8)
memory usage: 503.5+ KB


In [28]:
read_data = pd.get_dummies(read_data, columns = ['YearOfObservation'])

In [70]:
read_test = pd.get_dummies(read_test, columns = ['YearOfObservation'])

In [29]:
read_data['Date_of_Occupancy'] = 2019 - read_data['Date_of_Occupancy']  

In [71]:
read_test['Date_of_Occupancy'] = 2019 - read_test['Date_of_Occupancy']  

In [30]:
read_data['NumberOfWindows'].value_counts()

   .    3551
4        939
3        844
5        639
2        363
6        306
7        211
8        116
1         75
>=10      67
9         49
Name: NumberOfWindows, dtype: int64

In [31]:
read_data = pd.get_dummies(read_data, columns = ['NumberOfWindows'])

In [72]:
read_test = pd.get_dummies(read_test, columns = ['NumberOfWindows'])

In [32]:
for k in read_data.columns:
    read_data[k] = read_data[k].astype(np.float64)

In [73]:
for k in read_test.columns:
    read_test[k] = read_test[k].astype(np.float64)

In [33]:
read_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7160 entries, 0 to 7159
Data columns (total 30 columns):
Insured_Period            7160 non-null float64
Residential               7160 non-null float64
Building Dimension        7160 non-null float64
Building_Type             7160 non-null float64
Date_of_Occupancy         7160 non-null float64
Claim                     7160 non-null float64
Settlement_R              7160 non-null float64
Settlement_U              7160 non-null float64
Garden_O                  7160 non-null float64
Garden_V                  7160 non-null float64
Building_Fenced_N         7160 non-null float64
Building_Fenced_V         7160 non-null float64
Building_Painted_N        7160 non-null float64
Building_Painted_V        7160 non-null float64
YearOfObservation_2012    7160 non-null float64
YearOfObservation_2013    7160 non-null float64
YearOfObservation_2014    7160 non-null float64
YearOfObservation_2015    7160 non-null float64
YearOfObservation_2016    716

In [74]:
read_test.shape

(3069, 29)

In [34]:
target = read_data['Claim']
x = read_data.drop('Claim', axis = 1)

In [35]:
from sklearn.preprocessing import MinMaxScaler
min_max = MinMaxScaler()


In [94]:
x_max = min_max.fit_transform(x)
read_test = min_max.fit_transform(read_test)

In [37]:
x_max.shape

(7160, 29)

In [38]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(x_max, target, test_size = 0.2, random_state = 42, stratify = target)

In [39]:
x_train.shape, x_test.shape, y_train.shape, y_test.shape

((5728, 29), (1432, 29), (5728,), (1432,))

In [40]:
import catboost, lightgbm, xgboost

In [88]:
from xgboost import XGBClassifier
xgb = XGBClassifier()
xgb.fit(x_train, y_train)

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, gamma=0,
              learning_rate=0.1, max_delta_step=0, max_depth=3,
              min_child_weight=1, missing=None, n_estimators=100, n_jobs=1,
              nthread=None, objective='binary:logistic', random_state=0,
              reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
              silent=None, subsample=1, verbosity=1)

In [89]:
from sklearn.metrics import auc, classification_report, confusion_matrix

In [90]:
prediction = xgb.predict(x_test)

In [95]:
cross_pred = xgb.predict(read_test)
# export prediction 
ross_dict = {'Customer Id': customer_id, 'Claim':cross_pred}
frame = pd.DataFrame(ross_dict)
frame.to_csv('xgb.csv', index = False)

In [96]:
confusion_matrix(y_test, prediction)

array([[1061,   44],
       [ 260,   67]], dtype=int64)

In [97]:
from sklearn import metrics
fpr, tpr, thresholds = metrics.roc_curve(y_test, prediction.astype(np.float64))
auc(fpr, tpr)

0.5825369809179848

In [98]:
from sklearn.model_selection import cross_val_score
clf = XGBClassifier()
clf.fit(x_train, y_train)
scores = cross_val_score(clf, x_test, y_test, cv=10, scoring = 'roc_auc')
scores                                              


array([0.7048867 , 0.67431067, 0.60551461, 0.72781873, 0.69355719,
       0.63870523, 0.75964187, 0.64744318, 0.66178977, 0.66079545])

In [101]:
np.mean(scores)

0.6774463422616832

In [99]:
from sklearn.model_selection import cross_val_predict
predict = cross_val_predict(clf, x_test, y_test, cv = 10)

In [100]:
from sklearn.model_selection import cross_val_predict

cross_pred = xgb.predict(read_test)
# export prediction 
ross_dict = {'Customer Id': customer_id, 'Claim':cross_pred}
frame = pd.DataFrame(ross_dict)
frame.to_csv('xgb_cross.csv', index = False)

In [48]:
fpr, tpr, thresholds = metrics.roc_curve(y_test, predict.astype(np.float64))
auc(fpr, tpr)

0.571771347918137

In [49]:
from catboost import CatBoostClassifier

In [50]:
cat = CatBoostClassifier()
cat.fit(x_train, y_train)
prediction = cat.predict(x_test)
confusion_matrix(y_test, prediction)

Learning rate set to 0.027456
0:	learn: 0.6817945	total: 1.13s	remaining: 18m 52s
1:	learn: 0.6710868	total: 1.16s	remaining: 9m 38s
2:	learn: 0.6602274	total: 1.18s	remaining: 6m 32s
3:	learn: 0.6497088	total: 1.2s	remaining: 4m 59s
4:	learn: 0.6401966	total: 1.23s	remaining: 4m 4s
5:	learn: 0.6308900	total: 1.25s	remaining: 3m 27s
6:	learn: 0.6217052	total: 1.27s	remaining: 3m
7:	learn: 0.6132914	total: 1.29s	remaining: 2m 40s
8:	learn: 0.6062786	total: 1.31s	remaining: 2m 23s
9:	learn: 0.5993362	total: 1.33s	remaining: 2m 11s
10:	learn: 0.5925045	total: 1.35s	remaining: 2m 1s
11:	learn: 0.5859082	total: 1.38s	remaining: 1m 53s
12:	learn: 0.5795549	total: 1.4s	remaining: 1m 46s
13:	learn: 0.5743368	total: 1.43s	remaining: 1m 40s
14:	learn: 0.5689827	total: 1.46s	remaining: 1m 35s
15:	learn: 0.5634551	total: 1.52s	remaining: 1m 33s
16:	learn: 0.5583805	total: 1.54s	remaining: 1m 29s
17:	learn: 0.5536398	total: 1.57s	remaining: 1m 25s
18:	learn: 0.5492943	total: 1.6s	remaining: 1m 22s


array([[1047,   58],
       [ 260,   67]], dtype=int64)

In [51]:
from sklearn import metrics
prediction = cat.predict(x_test)
fpr, tpr, thresholds = metrics.roc_curve(y_test, prediction.astype(np.float64))
auc(fpr, tpr)

0.5762021392890254

In [52]:
from sklearn.model_selection import cross_val_score

scores = cross_val_score(cat, x_test, y_test, cv=3, scoring = 'roc_auc')
scores                                              


Learning rate set to 0.016532
0:	learn: 0.6863047	total: 6.82ms	remaining: 6.82s
1:	learn: 0.6788090	total: 20.2ms	remaining: 10.1s
2:	learn: 0.6716479	total: 59.9ms	remaining: 19.9s
3:	learn: 0.6641517	total: 98.9ms	remaining: 24.6s
4:	learn: 0.6572298	total: 166ms	remaining: 33s
5:	learn: 0.6504342	total: 182ms	remaining: 30.1s
6:	learn: 0.6444717	total: 200ms	remaining: 28.4s
7:	learn: 0.6383769	total: 287ms	remaining: 35.6s
8:	learn: 0.6324343	total: 300ms	remaining: 33s
9:	learn: 0.6269200	total: 345ms	remaining: 34.1s
10:	learn: 0.6211765	total: 382ms	remaining: 34.4s
11:	learn: 0.6157005	total: 407ms	remaining: 33.5s
12:	learn: 0.6106822	total: 421ms	remaining: 31.9s
13:	learn: 0.6054722	total: 447ms	remaining: 31.5s
14:	learn: 0.6003363	total: 468ms	remaining: 30.7s
15:	learn: 0.5962114	total: 478ms	remaining: 29.4s
16:	learn: 0.5916433	total: 498ms	remaining: 28.8s
17:	learn: 0.5874107	total: 533ms	remaining: 29.1s
18:	learn: 0.5837056	total: 549ms	remaining: 28.3s
19:	learn: 

array([0.61636956, 0.65565168, 0.66758077])

In [53]:
from sklearn.model_selection import cross_val_predict
predict = cross_val_predict(cat, x_test, y_test, cv = 3)

Learning rate set to 0.016532
0:	learn: 0.6863047	total: 4.3ms	remaining: 4.29s
1:	learn: 0.6788090	total: 58.8ms	remaining: 29.3s
2:	learn: 0.6716479	total: 71ms	remaining: 23.6s
3:	learn: 0.6641517	total: 120ms	remaining: 29.8s
4:	learn: 0.6572298	total: 172ms	remaining: 34.2s
5:	learn: 0.6504342	total: 186ms	remaining: 30.9s
6:	learn: 0.6444717	total: 231ms	remaining: 32.7s
7:	learn: 0.6383769	total: 295ms	remaining: 36.6s
8:	learn: 0.6324343	total: 360ms	remaining: 39.6s
9:	learn: 0.6269200	total: 374ms	remaining: 37s
10:	learn: 0.6211765	total: 405ms	remaining: 36.4s
11:	learn: 0.6157005	total: 472ms	remaining: 38.9s
12:	learn: 0.6106822	total: 495ms	remaining: 37.6s
13:	learn: 0.6054722	total: 578ms	remaining: 40.7s
14:	learn: 0.6003363	total: 663ms	remaining: 43.5s
15:	learn: 0.5962114	total: 672ms	remaining: 41.3s
16:	learn: 0.5916433	total: 742ms	remaining: 42.9s
17:	learn: 0.5874107	total: 754ms	remaining: 41.1s
18:	learn: 0.5837056	total: 768ms	remaining: 39.6s
19:	learn: 0.

In [54]:
fpr, tpr, thresholds = metrics.roc_curve(y_test, predict.astype(np.float64))
auc(fpr, tpr)

0.5715679355722529

In [55]:
from vecstack import stacking

In [57]:
models = [CatBoostClassifier(random_state= 0, learning_rate = 0.1,
                            n_estimators = 500),
        XGBClassifier(random_state=0, learning_rate=0.1, 
                  n_estimators=100, max_depth=3)
]

In [61]:
from sklearn.metrics import accuracy_score
S_train, S_test = stacking(models,                   
                           x_train, y_train, x_test,   
                           regression=False, 
     
                           mode='oof_pred_bag', 
       
                           needs_proba=False,
         
                           save_dir=None, 
            
                           metric=accuracy_score, 
    
                           n_folds=3, 
                 
                           stratified=True,
            
                           shuffle=True,  
            
                           random_state=0,    
         
                           verbose=2)

task:         [classification]
n_classes:    [2]
metric:       [accuracy_score]
mode:         [oof_pred_bag]
n_models:     [2]

model  0:     [CatBoostClassifier]
0:	learn: 0.6537377	total: 9.96ms	remaining: 4.97s
1:	learn: 0.6183083	total: 48.2ms	remaining: 12s
2:	learn: 0.5894518	total: 64.3ms	remaining: 10.6s
3:	learn: 0.5656552	total: 119ms	remaining: 14.7s
4:	learn: 0.5460487	total: 152ms	remaining: 15.1s
5:	learn: 0.5316597	total: 175ms	remaining: 14.4s
6:	learn: 0.5211736	total: 189ms	remaining: 13.3s
7:	learn: 0.5100380	total: 209ms	remaining: 12.9s
8:	learn: 0.5011052	total: 228ms	remaining: 12.4s
9:	learn: 0.4951346	total: 253ms	remaining: 12.4s
10:	learn: 0.4899548	total: 276ms	remaining: 12.3s
11:	learn: 0.4865402	total: 286ms	remaining: 11.6s
12:	learn: 0.4825667	total: 344ms	remaining: 12.9s
13:	learn: 0.4795903	total: 360ms	remaining: 12.5s
14:	learn: 0.4762869	total: 389ms	remaining: 12.6s
15:	learn: 0.4734201	total: 413ms	remaining: 12.5s
16:	learn: 0.4701851	total: 45

In [63]:
model = XGBClassifier(random_state=0, n_jobs=-1, learning_rate=0.1, 
                      n_estimators=100, max_depth=3)
    
model = model.fit(S_train, y_train)
y_pred = model.predict(S_test)
print('Final prediction score: [%.8f]' % accuracy_score(y_test, y_pred))


Final prediction score: [0.78421788]


In [64]:
fpr, tpr, thresholds = metrics.roc_curve(y_test,y_pred.astype(np.float64))
auc(fpr, tpr)

0.5641260879792991